In [1]:
# 1. Instalamos la librería Faker (si no está)
!pip install faker -q

from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, DateType
from faker import Faker
import random
from datetime import datetime, timedelta

# --- CONFIGURACIÓN ---
CANTIDAD_CLIENTES = 500
CANTIDAD_HABITACIONES = 50
CANTIDAD_CONTRATOS = 2000 
fake = Faker('es_ES')

# Configuración de Conexión
DB_HOST = "sql_server"
DB_PORT = "1433"
DB_USER = "sa"
DB_PASS = "PasswordFuerte123!" 
DRIVER_PKG = "com.microsoft.sqlserver:mssql-jdbc:11.2.0.jre8"
DRIVER_CLASS = "com.microsoft.sqlserver.jdbc.SQLServerDriver"
DB_NAME = "alquiler_habitacion"

print("--- INICIANDO GENERACIÓN MASIVA DE DATOS (CON LIMPIEZA) ---")

# Iniciar Spark
spark = SparkSession.builder \
    .appName("GeneradorDatosMasivos") \
    .master("local[*]") \
    .config("spark.jars.packages", DRIVER_PKG) \
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

# --- CORRECCIÓN CRÍTICA: REGISTRAR DRIVER MANUALMENTE ---
# Esto hace que 'DriverManager' encuentre el driver para los DELETE
try:
    spark.sparkContext._gateway.jvm.java.lang.Class.forName(DRIVER_CLASS)
    print("✅ Driver JDBC registrado en la JVM para comandos SQL puros.")
except Exception as e:
    print(f"⚠️ Advertencia registrando driver: {e}")

# 1. Función para ejecutar SQL Puro (DELETE, UPDATE, etc.)
def ejecutar_sql_puro(query):
    url = f"jdbc:sqlserver://{DB_HOST}:{DB_PORT};databaseName={DB_NAME};encrypt=true;trustServerCertificate=true;"
    try:
        manager = spark.sparkContext._gateway.jvm.java.sql.DriverManager
        con = manager.getConnection(url, DB_USER, DB_PASS)
        stmt = con.createStatement()
        stmt.execute(query)
        stmt.close()
        con.close()
        print(f"   ✅ Ejecutado: {query}")
    except Exception as e:
        print(f"   ❌ Error SQL: {e}")

# 2. Función auxiliar para escribir DataFrames
def guardar_en_sql(df, tabla):
    url = f"jdbc:sqlserver://{DB_HOST}:{DB_PORT};databaseName={DB_NAME};encrypt=true;trustServerCertificate=true;"
    print(f"   -> Insertando lotes en {tabla}...")
    df.write \
        .format("jdbc") \
        .option("url", url) \
        .option("dbtable", tabla) \
        .option("user", DB_USER) \
        .option("password", DB_PASS) \
        .option("driver", DRIVER_CLASS) \
        .mode("append") \
        .save()

# --- A. LIMPIEZA DE DATOS (Orden inverso para FKs) ---
print("\n>>> 1. Limpiando datos antiguos...")
ejecutar_sql_puro("DELETE FROM PAGO")
ejecutar_sql_puro("DELETE FROM CONTRATO")
ejecutar_sql_puro("DELETE FROM CLIENTE")
ejecutar_sql_puro("DELETE FROM HABITACION")
# PROPIETARIO no se borra para mantener el ID único si ya existe

# --- B. GESTIÓN DE PROPIETARIO ÚNICO ---
print("\n>>> 2. Verificando Propietario...")

url_read = f"jdbc:sqlserver://{DB_HOST}:{DB_PORT};databaseName={DB_NAME};encrypt=true;trustServerCertificate=true;"
try:
    df_props_existentes = spark.read.format("jdbc") \
        .option("url", url_read).option("dbtable", "PROPIETARIO") \
        .option("user", DB_USER).option("password", DB_PASS).option("driver", DRIVER_CLASS) \
        .load()
    count_props = df_props_existentes.count()
except:
    count_props = 0

if count_props > 0:
    print(f"   ℹ️ Ya existen {count_props} propietario(s). Usaremos el existente.")
else:
    print("   ✨ No hay propietarios. Creando Propietario Único...")
    propietarios = [(
        str(random.randint(10000000, 99999999)),
        "Juan Dueño Único", 
        fake.phone_number()[:20],
        fake.email()
    )]
    schema_prop = StructType([
        StructField("dniPropietario", StringType(), True),
        StructField("nombrePropietario", StringType(), True),
        StructField("celularPropietario", StringType(), True),
        StructField("correoPropietario", StringType(), True)
    ])
    df_prop = spark.createDataFrame(propietarios, schema=schema_prop)
    guardar_en_sql(df_prop, "PROPIETARIO")


# --- C. GENERAR HABITACIONES ---
print(f"\n>>> 3. Generando {CANTIDAD_HABITACIONES} habitaciones...")
tipos = ['Simple', 'Doble', 'Matrimonial', 'Suite', 'Penthouse']
habitaciones = []
for i in range(1, CANTIDAD_HABITACIONES + 1):
    numero = f"{random.randint(1, 10)}{random.randint(0, 9):02d}"
    habitaciones.append((
        numero, 
        random.choice(tipos), 
        float(random.choice([400, 500, 800])),
        'Disponible', 
        random.choice(['Wifi', 'Wifi, TV', 'Full'])
    ))

schema_hab = StructType([
    StructField("numeroHabitacion", StringType(), True),
    StructField("tipoHabitacion", StringType(), True),
    StructField("precioMensual", DoubleType(), True),
    StructField("estado", StringType(), True),
    StructField("serviciosIncluidos", StringType(), True)
])
df_hab = spark.createDataFrame(habitaciones, schema=schema_hab)
guardar_en_sql(df_hab, "HABITACION")


# --- D. GENERAR CLIENTES ---
print(f"\n>>> 4. Generando {CANTIDAD_CLIENTES} clientes...")
clientes = []
for _ in range(CANTIDAD_CLIENTES):
    clientes.append((
        fake.name(), 
        str(random.randint(10000000, 99999999)),
        fake.phone_number()[:20],
        fake.email(),
        fake.address()[:200]
    ))

schema_cli = StructType([
    StructField("nombreCli", StringType(), True),
    StructField("dniCli", StringType(), True),
    StructField("telefonoCli", StringType(), True),
    StructField("correoCli", StringType(), True),
    StructField("direccionCli", StringType(), True)
])
df_cli = spark.createDataFrame(clientes, schema=schema_cli)
guardar_en_sql(df_cli, "CLIENTE")


# --- E. GENERAR CONTRATOS ---
print(f"\n>>> 5. Preparando {CANTIDAD_CONTRATOS} contratos...")

# Recuperamos IDs reales
def obtener_ids(tabla, columna):
    df = spark.read.format("jdbc").option("url", url_read).option("dbtable", tabla).option("user", DB_USER).option("password", DB_PASS).option("driver", DRIVER_CLASS).load()
    return [row[0] for row in df.select(columna).collect()]

ids_clientes = obtener_ids("CLIENTE", "idCliente")
ids_habitaciones = obtener_ids("HABITACION", "idHabitacion")
ids_propietarios = obtener_ids("PROPIETARIO", "idPropietario")

if not ids_propietarios:
    raise Exception("❌ ERROR: No se encontró el ID del propietario. Algo falló en el paso 2.")

id_prop_unico = ids_propietarios[0]

contratos = []
fecha_base = datetime.now() - timedelta(days=730)

for _ in range(CANTIDAD_CONTRATOS):
    fecha_inicio = fecha_base + timedelta(days=random.randint(0, 700))
    monto = float(random.choice([400, 500, 600, 850, 1200]))
    
    contratos.append((
        fecha_inicio.date(),
        None, 
        monto,
        int(random.choice(ids_clientes)),
        int(random.choice(ids_habitaciones)),
        int(id_prop_unico), # Todos los contratos van al mismo propietario
        'Activo'
    ))

schema_contrato = StructType([
    StructField("fechaInicio", DateType(), True),
    StructField("fechaFin", DateType(), True),
    StructField("montoTotal", DoubleType(), True),
    StructField("idCliente", IntegerType(), True),
    StructField("idHabitacion", IntegerType(), True),
    StructField("idPropietario", IntegerType(), True),
    StructField("EstadoContrato", StringType(), True)
])
df_contratos = spark.createDataFrame(contratos, schema=schema_contrato)
guardar_en_sql(df_contratos, "CONTRATO")


# --- F. GENERAR PAGOS ---
print("\n>>> 6. Generando pagos históricos...")

# Leemos los contratos recién insertados
df_contratos_reales = spark.read.format("jdbc") \
    .option("url", url_read).option("dbtable", "CONTRATO") \
    .option("user", DB_USER).option("password", DB_PASS).option("driver", DRIVER_CLASS) \
    .load() \
    .select("idContrato", "fechaInicio", "montoTotal").collect()

pagos = []
for row in df_contratos_reales:
    id_contrato = row['idContrato']
    fecha_inicio = row['fechaInicio']
    monto = row['montoTotal']
    
    cant_pagos = random.randint(1, 12)
    for p in range(cant_pagos):
        if isinstance(fecha_inicio, datetime):
            f_pago = fecha_inicio + timedelta(days=30 * (p+1))
        else:
            f_pago = datetime.combine(fecha_inicio, datetime.min.time()) + timedelta(days=30 * (p+1))
            
        if f_pago > datetime.now(): break
        
        pagos.append((
            f_pago.date(),
            float(monto),
            random.choice(['Yape', 'Efectivo', 'Transferencia']),
            int(id_contrato)
        ))

print(f"   -> Insertando {len(pagos)} pagos...")
schema_pago = StructType([
    StructField("fechaPago", DateType(), True),
    StructField("montoPago", DoubleType(), True),
    StructField("metodoPago", StringType(), True),
    StructField("idContrato", IntegerType(), True)
])

# Batch processing para evitar saturar memoria
if pagos:
    df_pagos = spark.createDataFrame(pagos, schema=schema_pago)
    guardar_en_sql(df_pagos, "PAGO")
else:
    print("   ⚠️ No se generaron pagos (revisar fechas).")

print("\n✅ ¡Base de datos regenerada correctamente!")

--- INICIANDO GENERACIÓN MASIVA DE DATOS (CON LIMPIEZA) ---
⚠️ Advertencia registrando driver: An error occurred while calling z:java.lang.Class.forName.
: java.lang.ClassNotFoundException: com.microsoft.sqlserver.jdbc.SQLServerDriver
	at java.base/jdk.internal.loader.BuiltinClassLoader.loadClass(BuiltinClassLoader.java:641)
	at java.base/jdk.internal.loader.ClassLoaders$AppClassLoader.loadClass(ClassLoaders.java:188)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:525)
	at java.base/java.lang.Class.forName0(Native Method)
	at java.base/java.lang.Class.forName(Class.java:375)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:77)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.base/java.lang.reflect.Method.invoke(Method.java:568)
	at py4j.reflection.MethodInvoker.invoke(Metho